# Manipulating tensor shapes

In [1]:
import torch

## 1. tensor的存储方式

### 1.1 tensor.storage()
输出tensor存在memory中的1D data sequence

In [2]:
x = torch.tensor([[1, 2, 3], [6, 7, 8]])
x.storage()

/tmp/ipykernel_4246/806302124.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  x.storage()


 1
 2
 3
 6
 7
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]

### 1.2 tensor.layout attribute
1. 根据存储方式的不同，tensor有两种类型，dense tensor和sparse tensor，他们可以通过tensor.layout attribute来区分\
(1)**dense tensor** is stored linearly in a contiguous block of memory，也就是其存储占用了连续的memory block。\
(2)**sparse tensor**中大部分是0，只有很少比例的值是非零值，为了提高存储效率，pytorch为这类数据提供了特殊的存储方式。\
2. torch.layout attribute是表达tensor的memory layout特征的object。这个attribute有两种类型的取值：torch.stride和torch.sparse_coo，分别对应上面的两种tensor类型\
(1)如果<font color=blue>**layout=torch.strided**</font>，那么该tensor是dense tensor\
(2)如果<font color=blue>**layout=torch.sparse**</font>，那么该tensor是sparse tensor

In [3]:
# 新建一个dense tensor
x = torch.rand((2, 3), layout=torch.strided, requires_grad=True)
y = torch.zeros((100, 200), layout=torch.sparse_coo, requires_grad=False)
print("dense tensor x:\n", x)
print(x.is_contiguous())        # x是连续存储
print("sparse tensor y:\n", y)
print(y.is_contiguous())        # y不是连续存储

dense tensor x:
 tensor([[0.4545, 0.6053, 0.2413],
        [0.5411, 0.7210, 0.6547]], requires_grad=True)
True
sparse tensor y:
 tensor(indices=tensor([], size=(2, 0)),
       values=tensor([], size=(0,)),
       size=(100, 200), nnz=0, layout=torch.sparse_coo)
False


## 2. 改变dense tensor的view

### 2.1 dense tensor的contiguous特征
1. dense tensor可能是contiguous tensor或者non-contiguous tensor。虽然他们在存储空间中都占用了连续的存储单元，但non-contiguous tensor的元素排列顺序与其元素在存储空间的排列顺序不一致。\
**· <font color=blue>tensor.view()</font>** 会保持contiguous tensor的contiguity。也只有contiguous tensor有view() method\
**· <font color=blue>tensor.transpose()</font>** 会改变tensor的contiguity。contiguous和non-contiguous tensor都可以transpose。当base tensor是dense tensor时，transpose不copy data，所以output tensor也只是原underlying data的一个new view。
2. 用is_contiguous()可以查看是否contiguous
3. 用tensor.contiguous()可以把non-contiguous tensor转变成contiguous

In [4]:
base = torch.tensor([[0, 1],[2, 3]])
print(base.is_contiguous(), base.storage().data_ptr())

t = base.transpose(0, 1) 
print(t.is_contiguous(), t.storage().data_ptr())  # 同样的data
print(base.is_contiguous())                       # contiguity变化

True 104839808
False 104839808
True


In [5]:
## 用tensor.storage()可以获得该tensor在memory上的1D sequence
x = torch.arange(0,12)
print(x.storage(), '\n')

## view返回的tensor中，data sequence的排序和原数据相同，都是0, 1, 2, 3, 4...
x = x.view(2,2,3)
print('x: \n', x)
print(x.stride(), x.is_contiguous(), '\n')

## transpose返回的tensor中，data sequence的排序和原数据在memory中的排序不同
#  y中元素的排序是：0, 1, 2, 6, 7...
#  transpose不copy data，它仍然是原data的view，但是是non-contiguous ‘View’
#  按新tensor的element顺序从memory中取值的时候无法连续获得这些值
y = x.transpose(0,1)
print('y: \n', y)
print(y.stride(), y.is_contiguous())

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12] 

x: 
 tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]]])
(6, 3, 1) True 

y: 
 tensor([[[ 0,  1,  2],
         [ 6,  7,  8]],

        [[ 3,  4,  5],
         [ 9, 10, 11]]])
(3, 6, 1) False


In [6]:
## transpose将non-contiguous转变成contiguous
w = y.transpose(0, 1)
w.is_contiguous()

True

In [7]:
## 用tensor.contiguous()转变成contiguous tensor
#  此时发生了copy，按照tensor view的排序方式复制了一个新的data
z = y.contiguous()
print('z: \n', z)
print(z.stride(), z.is_contiguous())
print(z.storage())

z: 
 tensor([[[ 0,  1,  2],
         [ 6,  7,  8]],

        [[ 3,  4,  5],
         [ 9, 10, 11]]])
(6, 3, 1) True
 0
 1
 2
 6
 7
 8
 3
 4
 5
 9
 10
 11
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]


### 2.2 tensor.view() method
<font color=red>相当于numpy里面的reshape。</font>
1. 只有contiguous dense tensor才有view method。
2. dense tensor在计算机中以1D data sequence的形式存在congtiguous block of memory上。tensor.view()本质上是告诉计算机如何(根据其参数设置)stride over the 1D data sequence。所以，只提供了new view of the base tensor，不改变底层data。view的改变也就意味着tensor.stride()也会随之改变。
3. tensor.view()返回的new tensor从memory上读取elements的顺序与其线性存储的顺序一致，只是改变了划分layer,column和row等unit单位的位置。
4. 因为View tensor与它的base tensor的underlying data是同一个data，即tensor.view()返回new tensor，但并不copy data，所以，其中一个tensor元素值改变，另一个也变。

**tensor.data_ptr()**: 返回1st element的存储地址，即data pointer指向的值

In [8]:
## tensor.view()返回的新tensor不copy data，但是会改变stride和shape(size)
a = torch.arange(15)
print('原data pointer:', a.data_ptr()) # 
print('原id:', id(a))
print('原stride和size:', a.stride(), a.size(), '\n')

# change view
b = a.view(3, 5)
print('新data pointer与原值相同:', b.data_ptr())
print('id变化，说明新建了一个tensor:', id(b))                 # id变化
print('stride和size变化:', b.stride(), b.size()) 

# data地址不变
x.untyped_storage().data_ptr() == y.untyped_storage().data_ptr()

原data pointer: 104895232
原id: 140235751104304
原stride和size: (1,) torch.Size([15]) 

新data pointer与原值相同: 104895232
id变化，说明新建了一个tensor: 140235751104464
stride和size变化: (5, 1) torch.Size([3, 5])


True

In [9]:
## tensor.view()不改变tensor layout in memory
a = torch.rand(18)
print("原shape:", '\n', a)
print(a.size(), id(a), '\n')

b = a.view(1, 3, 2, 3)
print("用view不改变元素在memory中的排序:", '\n', b)
print(b.size(), id(b), '\n')

c = a.view(3, 6)  # 不改变tensor layout in memory
print("用view不改变元素在memory中的排序:", '\n', c)
print(c.size(), id(c), '\n')

# c和b只是shape相同，但是元素值不同
torch.equal(b, c), c is a

原shape: 
 tensor([0.2335, 0.1410, 0.4894, 0.9493, 0.9713, 0.8237, 0.8002, 0.1175, 0.6101,
        0.2457, 0.7504, 0.3586, 0.4528, 0.6221, 0.6928, 0.4970, 0.2736, 0.9745])
torch.Size([18]) 140235751106784 

用view不改变元素在memory中的排序: 
 tensor([[[[0.2335, 0.1410, 0.4894],
          [0.9493, 0.9713, 0.8237]],

         [[0.8002, 0.1175, 0.6101],
          [0.2457, 0.7504, 0.3586]],

         [[0.4528, 0.6221, 0.6928],
          [0.4970, 0.2736, 0.9745]]]])
torch.Size([1, 3, 2, 3]) 140235771701952 

用view不改变元素在memory中的排序: 
 tensor([[0.2335, 0.1410, 0.4894, 0.9493, 0.9713, 0.8237],
        [0.8002, 0.1175, 0.6101, 0.2457, 0.7504, 0.3586],
        [0.4528, 0.6221, 0.6928, 0.4970, 0.2736, 0.9745]])
torch.Size([3, 6]) 140235751104944 



(False, False)

### 2.3 tensor.stride() method
1. dense tensor才有**tensor.stride()**。contiguous和non-contiguous tensor也都有stride。
2. strided tensor的stride是一个int list,其第k-th元素的值表示tensor的k-th dimension上，一个元素到下个元素之间，用units of elements衡量的长度。\
· the k-th stride represents the jump in the memory necessary to go from one element to the next one in the k-th dimension of the Tensor.
3. 每个strided tensor都有一个与之关联的torch.Storage存放data. 这些tensors 则提供strided view of a storage。一个torch.Storage可以对应不同的strided tensors。这些tensors有相同的data，不同的tensor.view()\
<font color=blue>**· Numpy strides()** returns (N bytes to Next Row, M bytes to Next Column)\
**· Pytorch stride()** returns (N elements to Next Row, M elements to Next Column).</font>

In [10]:
x = torch.tensor([[1, 2, 3, 4, 5],
                  [6, 7, 8, 9, 10]])

print(x.stride())  # 不指定dim，A tuple of all strides is returned
                   # 在tuple(5, 1)中，5在第0维，1在第1维对应traverse的维度
print(x.stride(0)) # 指定dim=0，返回该维度上的stride值5
                   # traverse along the 0th dim, 比如从'1'跳到'6',距离是5
print(x.stride(-1))# 指定dim=1，返回该维度上的stride值1
                   # traverse along the 1st dim, 比如从'7'跳到'8',距离是1

## 改变view，stride也改变
y = x.view(5, 2)
print(y.stride())

(5, 1)
5
1
(2, 1)


### 2.4 tensor.transpose() method
1. 和view method不同的是，transpose的output tensor与input tensor的shape不兼容。也就是改变了elements在view中的layout。如果把transpose之后的tensor排成一个vector，其顺序与input tensor不相同。
2. 和view method相同的是，当tensor是dense tensor时，output只是input的一个view，不发生copy。改变其中一个的元素值，另一个也变

In [11]:
a = torch.arange(24).reshape(1, 2, 3, 4)
print("原shape:", '\n', a)
print(a.size(), id(a), '\n')

b = a.transpose(1, 2)
print("用transpose交换了2nd和3rd dim:", '\n', b)
print(b.size(), id(b), '\n')

# b.view(24) # 错，b这时不是contiguous，不能用view method

原shape: 
 tensor([[[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11]],

         [[12, 13, 14, 15],
          [16, 17, 18, 19],
          [20, 21, 22, 23]]]])
torch.Size([1, 2, 3, 4]) 140235751105824 

用transpose交换了2nd和3rd dim: 
 tensor([[[[ 0,  1,  2,  3],
          [12, 13, 14, 15]],

         [[ 4,  5,  6,  7],
          [16, 17, 18, 19]],

         [[ 8,  9, 10, 11],
          [20, 21, 22, 23]]]])
torch.Size([1, 3, 2, 4]) 140239248608928 



### 2.5 squeeze and unsqueeze
· squeeze和unsqueeze只改变原数据的view，新生成的tensor与input tensor指向相同的memory

**2.5.1 unsqueeze**
1. pytorch默认是按照batch处理数据，以image为例，默认的input大小是(N, C, H, W)。如果想让model处理单个数据，就要把当个样本的大小从(C, H, W)改成(1, C, H, W)
2. 常用于ease broadcast

In [12]:
# 用tensor.unsqueeze()来增加长度为1的新维度
a = torch.rand(3, 226, 226)
b = a.unsqueeze(0) # 在第0维增加一个维度
print('增加一个长度为1的维度到新的第0维：',b.shape)

c = a.unsqueeze(1)
print('增加一个长度为1的维度到新的第1维：',c.shape)

增加一个长度为1的维度到新的第0维： torch.Size([1, 3, 226, 226])
增加一个长度为1的维度到新的第1维： torch.Size([3, 1, 226, 226])


In [13]:
# unsqueeze常用于方便broadcast
a = torch.ones(4, 3, 2)
b = torch.rand(   3)     # a * b不能直接运算
c = b.unsqueeze(1)       # change to a 2-dimensional tensor, adding new dim at the end
print(c.shape)
print(a * c)             # broadcasting works again!

torch.Size([3, 1])
tensor([[[0.9940, 0.9940],
         [0.6353, 0.6353],
         [0.0612, 0.0612]],

        [[0.9940, 0.9940],
         [0.6353, 0.6353],
         [0.0612, 0.0612]],

        [[0.9940, 0.9940],
         [0.6353, 0.6353],
         [0.0612, 0.0612]],

        [[0.9940, 0.9940],
         [0.6353, 0.6353],
         [0.0612, 0.0612]]])


**2.5.2 squeeze**
1. 用tensor.squeeze()来压缩长度为1的维度(dimensions of extent 1)
2. 最好指定dim number，因为pytorch按batch处理数据，squeeze()会把第1维N压掉，导致模型出错 

In [14]:
a = torch.ones(1, 2, 1, 4)
b = a.squeeze()   # 压缩所有维度为1的dims
c = a.squeeze(0)  # 压缩第0维
print(b.shape)
print(c.shape)

## 改变b之后，a的值也变
b += 1
print(a)
print(a.storage())
print(a.stride())

torch.Size([2, 4])
torch.Size([2, 1, 4])
tensor([[[[2., 2., 2., 2.]],

         [[2., 2., 2., 2.]]]])
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 8]
(8, 4, 4, 1)


In [15]:
x = torch.tensor([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]])
print(x.stride(), x.T.stride())

(5, 1) (1, 5)


## 3. reshape工具

### 3.1 只改变view，不copy data的reshape工具
1. tensor.view
2. tensor.transpose
3. tensor.squeeze and tensor.unsqueeze

### 3.2 tensor.reshape()
1. 通常情况下，reshape返回的tensor只是原tensor的一个view，两者指向的是相同的memory location。但实际使用的时候，不要依赖这里的view vs. copy关系，不然容易出错。\
2. reshape实际上是打包了两种处理方式的method，当reshape的input tensor和outpu tensor的shape是compatible的时候，它调用的是tensor.view()，此时不copy data，当两者的shape不兼容的时候，它调用tensor.contiguous()，这时候就会copy data。\
<font color=blue>· 可以理解成，处理contiguous tensor时，调用view();处理non-contiguous tensor时，调用contiguous(). </font>
3. pyrotch要求'shape'参数必须是tuple of ints。但是当shape是第一个参数的时候，可以用series if integers或者单个integer作为shape的值，但如果shape不是第一个参数，就必须是tuple。\
· 注：在extent of dim为1时，要注意，x=(3)是int，x=(3,)是tuple

In [16]:
## tensor.reshape()的shape参数：x=(3)是int，不能用，x=(3,)是tuple
x = (1, 2, 3)
y = (3)
z = (2,)
print(type(x), type(y), type(z))

## 用tensor.reshape,这时shape参数是第一个参数
output3d = torch.ones(2, 2, 3)

input1d = output3d.reshape(12) # 12 = 2 * 2 * 3
print(input1d, input1d.shape)

## 用torch module的reshape method，这时shape参数不是第一个参数
# input1d_2 = torch.reshape(output3d, (12)) # 错,(12)被识别为int12
input1d_2 = torch.reshape(output3d, (12,))  # 用(12,)才是tuple
print(input1d_2.shape)

<class 'tuple'> <class 'int'> <class 'tuple'>
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]) torch.Size([12])
torch.Size([12])


In [17]:
## 处理contiguous tensor
x = torch.arange(1,13)
print(x, x.is_contiguous())

y = x.reshape(4,3)
print(y)
print(x.storage().data_ptr() == y.storage().data_ptr())

# 改变x中元素值, y也变
x[2] = 100
print(x, '\n')
print(y, '\n')

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]) True
tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]])
True
tensor([  1,   2, 100,   4,   5,   6,   7,   8,   9,  10,  11,  12]) 

tensor([[  1,   2, 100],
        [  4,   5,   6],
        [  7,   8,   9],
        [ 10,  11,  12]]) 



In [18]:
## 处理non-contiguous tensor
x = torch.arange(1,13).view(6,2).transpose(0,1)
print(x, x.is_contiguous())

y = x.reshape(4,3)
print(y)
print(x.storage().data_ptr() == y.storage().data_ptr())

# 改变x中元素值, y不变
x[1] = 100
print(x, '\n')
print(y, '\n')

tensor([[ 1,  3,  5,  7,  9, 11],
        [ 2,  4,  6,  8, 10, 12]]) False
tensor([[ 1,  3,  5],
        [ 7,  9, 11],
        [ 2,  4,  6],
        [ 8, 10, 12]])
False
tensor([[  1,   3,   5,   7,   9,  11],
        [100, 100, 100, 100, 100, 100]]) 

tensor([[ 1,  3,  5],
        [ 7,  9, 11],
        [ 2,  4,  6],
        [ 8, 10, 12]]) 

